In [ ]:
%run -i  'run_elmo_head.py'

In [ ]:
import torch
from torchviz import make_dot

In [ ]:
model=torch.nn.Sequential()
model.add_module('W0', torch.nn.Linear(8,16))
model.add_module('tanh', torch.nn.Tanh())
model.add_module('W1', torch.nn.Linear(16,1))

x=torch.randn(1,8)
y=model(x)
print(x)
print(y)
make_dot(y.mean(), params=dict(model.named_parameters()))

In [ ]:
import argparse
from factslab.utility import read_data, interleave_lists
from factslab.pytorch.mlpregression import MLPTrainer, MLPRegression
from torch.cuda import is_available
from torch import device
from os.path import expanduser
import pickle
from torch.nn import Module, Linear, ModuleDict, ModuleList
from torch.nn import MSELoss, L1Loss, SmoothL1Loss, CrossEntropyLoss, NLLLoss
import numpy as np
from sklearn.metrics import accuracy_score as acc, f1_score as f1, precision_score as prec, recall_score as rec, r2_score as r2, mean_squared_error as mse
import torch
import torch.nn.functional as F
from scipy.stats import mode
from collections import defaultdict
from functools import partial
from allennlp.commands.elmo import ElmoEmbedder
from os.path import expanduser
from tqdm import tqdm
from graphviz import Digraph
from torch.autograd import Variable
from torchviz import make_dot

In [ ]:
train_in = open("train_stuff.pkl", "rb")
dev_in = open("dev_stuff.pkl", "rb")
train_data = pickle.load(train_in)
dev_data = pickle.load(dev_in)
train_in.close()
dev_in.close()

In [ ]:
# ELMO parameters
options_file = '/srv/models/pytorch/elmo/' + "options/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json"
weight_file = '/srv/models/pytorch/elmo/' + "weights/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5"
elmo_params = (options_file, weight_file)

# pyTorch figures out device to do computation on
device_to_use = device("cuda:0" if is_available() else "cpu")
model_type = {'arg': {'repr': 'param', 'context': 'david'},
                  'pred': {'repr': 'param', 'context': 'david'}}
attributes = {'arg': ['part', 'kind', 'abs'], 'pred': ['part', 'dyn', 'hyp']}

In [ ]:
x, y, tokens, spans, context_roots, context_spans, loss_wts = [item[0] for item in train_data]
x1, y1, tokens1, spans1, context_roots1, context_spans1, loss_wts1 = [item[1] for item in train_data]

In [ ]:
model = MLPRegression(output_size=3, device=device_to_use, attention_type=model_type, embed_params=elmo_params,
                     all_attrs=attributes)
model.to(device_to_use)

In [ ]:
prot="arg"
for attr in attributes[prot]:
    y[attr] = torch.tensor(y[attr], dtype=torch.long, device=device_to_use)

y_ = model(prot=prot, inputs=x, tokens=tokens, spans=spans, context_roots=context_roots, context_spans=context_spans)

In [ ]:
make_dot(y_['part'].mean(), params=dict(model.named_parameters()))

In [ ]:
prot="pred"
for attr in attributes[prot]:
    y1[attr] = torch.tensor(y1[attr], dtype=torch.long, device=device_to_use)

y1_ = model(prot=prot, inputs=x1, tokens=tokens1, spans=spans1, context_roots=context_roots1, context_spans=context_spans1)